In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import numpy as np
import pandas as pd
import collections

In [ ]:
x_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터3] samp_cst_feat'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

df = pd.merge(df_x, df_y, on='cst_id_di')

In [ ]:
df.MRC_ID_DI[df.MRC_ID_DI > 0 ] = 1

In [ ]:
nc_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터4] variable_dtype'}.xlsx")
nc = pd.read_excel(nc_name, index_col=0)

In [ ]:
X = df.drop(columns = ['MRC_ID_DI'], axis=1)
y = df['MRC_ID_DI']

In [ ]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

def add_interactions(df):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]
    
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis = 1)
    
    return df

X = add_interactions(X)

import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=331)
selected_features = select.fit(X, y)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X.columns[i] for i in indices_selected]

X = X[colnames_selected]

In [ ]:
X[X.columns] = (X[X.columns]+1)/2*255

In [ ]:
from PIL import Image
import numpy as np
for ind, k in enumerate(df.index):
    data = np.zeros((331, 331, 3), dtype=np.uint8)
    for i in range(331):
        for j in range(331):
            data[i, j] = [(X.iloc[ind, j] + X.iloc[ind, i])//2, (X.iloc[ind, j] + X.iloc[ind, i])//2, (X.iloc[ind, j] + X.iloc[ind, i])//2]
    img = Image.fromarray(data, 'RGB')
    img_name = os.path.join(os.getcwd(), 'image4', str(k) + '.png')
    img.save(img_name)